## Descripción del dataset: Pima Indians Diabetes

El **Pima Indians Diabetes Dataset** es un conjunto de datos clásico en Machine Learning y bioestadística, recopilado por el *National Institute of Diabetes and Digestive and Kidney Diseases*.  
Su propósito es **predecir la aparición de diabetes tipo 2** en mujeres de origen **pima** (una población indígena del sur de Arizona, EE.UU.), a partir de diversas variables clínicas y demográficas.

### Características principales:
- **Número de registros:** 392 (en esta versión limpia, el original tenía 768).  
- **Número de atributos (features):** 8 variables predictoras + 1 variable objetivo.  
- **Población:** Mujeres de al menos 21 años de edad de la etnia Pima.  
- **Tarea principal:** Clasificación binaria → determinar si una paciente tiene diabetes (`Outcome = 1`) o no (`Outcome = 0`).

### Variables:
1. **Pregnancies** → Número de embarazos.  
2. **Glucose** → Concentración de glucosa en plasma después de 2 horas en una prueba de tolerancia a la glucosa.  
3. **BloodPressure** → Presión arterial diastólica (mm Hg).  
4. **SkinThickness** → Espesor del pliegue cutáneo del tríceps (mm).  
5. **Insulin** → Nivel sérico de insulina (mu U/ml).  
6. **BMI** → Índice de masa corporal (peso en kg / altura² en m²).  
7. **DiabetesPedigreeFunction** → Probabilidad de diabetes basada en antecedentes familiares.  
8. **Age** → Edad en años.  
9. **Outcome** → Variable objetivo:  
   - `0` = No tiene diabetes  
   - `1` = Tiene diabetes  

### Relevancia:
Este dataset es ampliamente utilizado en cursos de **Inteligencia Artificial y Machine Learning** para enseñar:
- Procesamiento y limpieza de datos biomédicos.  
- Métodos de clasificación supervisada (KNN, regresión logística, Random Forest, SVM, redes neuronales, etc.).  
- Importancia de la normalización y estandarización en algoritmos basados en distancias.  

---


## Paso 1: Cargar la base de datos  
Cargamos el CSV en un `DataFrame` de `pandas`. Si tu archivo no se llama exactamente `cleaned_dataset.csv`, ajusta la ruta.

In [5]:
import pandas as pd
from pathlib import Path
import os

df = pd.read_csv("cleaned_dataset.csv")

display(df.head())


,Pregnancies,Glucose,Blood Pressure,Skin Thickness,Insulin,BMI,Diabetes Pedigree Function,Age,Outcome
0,0,129,110,46,130,67.1,0.319,26,1
1,0,180,78,63,14,59.4,2.420,25,1
2,3,123,100,35,240,57.3,0.880,22,0
3,1,88,30,42,99,55.0,0.496,26,1
4,0,162,76,56,100,53.2,0.759,25,1


## Paso 2: Crear subconjuntos con 20 datos de **entrenamiento** y 20 de **testeo**
Seleccionaremos 40 muestras: 20 para entrenar y 20 para evaluar.

In [6]:
from sklearn.model_selection import train_test_split

# Seleccionar 40 muestras del dataset cargado (20 para entrenamiento, 20 para testeo)
if 'df' not in locals():
    raise ValueError("El DataFrame 'df' no está definido. Asegúrate de ejecutar el Paso 1 antes de continuar.")

# Dividir el dataset en entrenamiento y testeo (20/20)
train, test = train_test_split(df, test_size=0.5, stratify=df['Outcome'], random_state=42)

# Seleccionar las primeras 20 muestras de cada conjunto
train_subset = train.iloc[:20]
test_subset = test.iloc[:20]

print("Conjunto de entrenamiento (20 muestras):")
print(train_subset)
print("\nConjunto de testeo (20 muestras):")
print(test_subset)

Conjunto de entrenamiento (20 muestras):
     Pregnancies  Glucose  Blood Pressure  Skin Thickness  Insulin   BMI  \
89             1       87              68              34       77  37.6   
173            8      155              62              26      495  34.0   
367            4       99              76              15       51  23.2   
296            1       89              66              23       94  28.1   
78             2      146              76              35      194  38.2   
52             1      128              48              45      194  40.5   
60             5      158              84              41      210  39.4   
30             1      103              30              38       83  43.3   
218            4      123              80              15      176  32.0   
93             0       95              85              25       36  37.4   
247            2      106              64              35      119  30.5   
91            10      148              84      

## Paso 3: Implementar la función de distancia euclidiana

**Instrucciones:**
- Escribe una función en Python que reciba dos vectores y calcule la distancia euclidiana entre ellos.
- Utiliza la siguiente fórmula matemática para la distancia euclidiana entre dos vectores $x$ y $y$ de $n$ dimensiones:

$$
d(x, y) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

- Prueba tu función con los siguientes dos ejemplos (cada vector corresponde a una fila del dataset):

| Embarazos | Glucosa | Presión Arterial | Grosor Piel | Insulina | IMC  | Función Hereditaria | Edad | Resultado |
|-----------|---------|------------------|-------------|----------|------|---------------------|------|-----------|
|     1     |   106   |        70        |      28     |   135    | 34.2 |        0.142        |  22  |     0     |
|     2     |   102   |        86        |      36     |   120    | 45.5 |        0.127        |  23  |     1     |

- Calcula la distancia euclidiana a mano y luego verifica que el resultado de tu función sea el mismo.
- La función debe imprimir el resultado del cálculo de la distancia euclidiana con los datos presentados.



In [7]:
import numpy as np

def euclidean_distance(vector1, vector2):
    """Calcula la distancia euclidiana entre dos vectores."""
    if len(vector1) != len(vector2):
        raise ValueError("Los vectores deben tener la misma longitud.")
    return np.sqrt(np.sum((np.array(vector1) - np.array(vector2)) ** 2))

# Ejemplo de prueba
vector_a = [1, 106, 70, 28, 135, 34.2, 0.142, 22]
vector_b = [2, 102, 86, 36, 120, 45.5, 0.127, 23]

distance = euclidean_distance(vector_a, vector_b)
print(f"La distancia euclidiana entre los vectores es: {distance:.4f}")

La distancia euclidiana entre los vectores es: 26.2810


## Paso 4: Implementar un clasificador KNN básico

**Instrucciones:**
- Escribe una función que, dado un punto de prueba, calcule la distancia a todos los puntos de entrenamiento utilizando tu función de distancia euclidiana.
- Selecciona los **k = 3** vecinos más cercanos y predice la clase mayoritaria entre ellos.
- Aplica tu función a las 10 muestras de prueba obtenidas previamente, utilizando las 10 muestras de entrenamiento como referencia.
- El script debe imprimir una tabla comparando el valor real de `Resultado` de cada muestra de prueba con el valor predicho por tu algoritmo.
- Considere que las tablas se pueden codificar con un formato similar al que se muestra en el siguiente código:

In [13]:
#codigo aqui

## Paso 5: Usar toda la data con separación 80% entrenamiento / 20% testeo  

### Pasos:
1. Cargar todo el dataset.  
2. Separar variables (X) y etiquetas (y).  
3. Aplicar `train_test_split` con 80% para entrenamiento y 20% para testeo.  
4. Mantener la proporción de clases usando estratificación.  
5. Guardar los conjuntos de datos para usarlos en KNN.  

In [ ]:
#codigo aqui s

## Paso 6: Entrenar un KNN con los datos sin escalar (crudos) y calcular accuracy  

### Pasos:
1. Definir el valor de **k = 3** y el metodo **Euclidiano**.  
2. Entrenar el modelo KNN con los datos crudos (sin normalizar/estandarizar).  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** comparando predicciones con etiquetas reales.  
5. Guardar el resultado para la tabla comparativa.  


In [15]:
#codigo aqui

## Paso 7: Normalizar (Min-Max scaling) y entrenar KNN, luego calcular accuracy  

### Pasos:
1. Aplicar **normalización Min-Max** a los datos de entrenamiento y test.  
2. Entrenar el modelo KNN con los datos normalizados.  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** del modelo.  
5. Guardar el resultado para la tabla comparativa.  


In [16]:
#codigo aqui

## Paso 9: Estandarizar (Z-score) y entrenar KNN, luego calcular accuracy  

### Pasos:
1. Aplicar **estandarización Z-score** a los datos de entrenamiento y test.  
2. Entrenar el modelo KNN con los datos estandarizados.  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** del modelo.  
5. Guardar el resultado para la tabla comparativa.  


In [17]:
#codigo aqui

## Paso 10/11: Tabla comparativa de accuracies  

### Pasos:
1. Reunir los resultados de accuracy de cada experimento:  
   - KNN sin escalar (80/20).  
   - KNN normalizado (80/20).  
   - KNN estandarizado (80/20).  
2. Crear una tabla con los resultados.  
3. Comparar el desempeño de cada método.  



In [18]:
#codigo aqui

---
## Preguntas de reflexión y aplicación



1. ¿Por qué es importante normalizar o estandarizar los datos antes de usar KNN?  



Responda aqui

2. ¿Qué diferencias observaste en el accuracy entre los datos crudos, normalizados y estandarizados?  


Respinda aqui

3. Si aumentamos el valor de **k** (número de vecinos), ¿cómo crees que cambiaría el rendimiento del modelo?  


Responda aqui

4. ¿Qué ventaja tiene implementar KNN manualmente antes de usar scikit-learn?  


respuesta aqui

5. ¿Qué limitaciones presenta KNN cuando se aplica a conjuntos de datos grandes o con muchas dimensiones?  

respuesta aqui

---

## Rúbrica de evaluación: Práctica KNN

| Criterio | Descripción | Puntaje Máximo |
|----------|-------------|----------------|
| **1. Carga y exploración del dataset** | Carga correcta del archivo CSV, explicación de las variables y verificación de datos. | 15 pts |
| **2. Implementación manual de KNN** | Código propio para calcular distancias euclidianas, selección de vecinos y votación mayoritaria. | 20 pts |
| **3. Predicción individual (ejemplo aleatorio)** | Explicación clara del proceso paso a paso para un ejemplo de test. | 10 pts |
| **4. Uso de scikit-learn (KNN)** | Entrenamiento y evaluación con `train_test_split`, comparación con el método manual. | 15 pts |
| **5. Normalización y estandarización** | Aplicación correcta de Min-Max y Z-score, con cálculo de accuracy en cada caso. | 20 pts |
| **6. Tabla comparativa de accuracies** | Presentación clara de los resultados y comparación entre métodos. | 10 pts |
| **7. Reflexión y preguntas finales** | Respuestas a las preguntas de análisis planteadas (profundidad y claridad). | 10 pts |

**Total: 100 pts**
